In [20]:
import numpy as np
import cv2
import os
import dlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

SIZE = (200,200)

i = 0
names = {}
for p in os.listdir(base_dir):
    names[p] = i
    i+=1


In [21]:
names

{'Hrishi': 0, 'Jenil': 1, 'saurabh': 2, 'Tomar': 3}

In [22]:
def preprocess_data(data, noise_remove=True, scaling=True):
    # standard scaling
    if scaling:
        data = np.array(data)
        mean = np.mean(data, axis=0)
        std = np.std(data, axis=0)
        data = (data-mean)/std
        
    # removing noise:
    if noise_remove:
        new_data = []
        for i in data:
            tmp = cv2.GaussianBlur(i,(3,3),0)
#             tmp = cv2.bilateralFilter(tmp, 15,75,75)
            new_data.append(tmp)
    data = np.array(new_data)
    
    return data

###################################
def make_dataset():
    base_dir = r"E:\semester 5\mini project\face recognition algorithms\dataset"
    training_images_dir = [base_dir+'\\'+name for name in os.listdir(base_dir)]
    names = {}
    i = 0
    for p in os.listdir(base_dir):
        names[p] = i
        i+=1
    X = []
    y = []
    for person_dir in training_images_dir:
        for img in os.listdir(person_dir):
            image = cv2.imread(person_dir+'\\'+img)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.resize(image, SIZE, interpolation=cv2.INTER_LINEAR)
            X.append(image)
            y.append(names[person_dir.split('\\')[-1]])
    data = np.array(X)
    labels = np.array(y)
    return data,labels

###################################
def get_faces_from_image_2(img):
    detector = dlib.get_frontal_face_detector()
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray_img)
    cropped_faces = []
    take = 0  # increase the size of the bounding box
    for face in faces:
        cropped_faces.append(img[ face.top()-take :face.bottom()+take, face.left()-take:face.right()+take , :].copy())
    return cropped_faces

In [23]:
x,y = make_dataset()
x_train ,x_test, y_train, y_test = train_test_split(x,y,test_size=0.10, random_state=60)
x_train = preprocess_data(x_train)
x_test = preprocess_data(x_test)

In [24]:
# if 'FISHERFACE' not in os.listdir(r'E:\semester 5\mini project\face recognition algorithms'):
face_recognizer = cv2.face.FisherFaceRecognizer_create()
face_recognizer.train(x_train,y_train)
face_recognizer.write('FISHERFACE')

# else :
#     face_recognizer = cv2.face.FisherFaceRecognizer_create()
#     face_recognizer.read('FISHERFACE')


predictions = []
for i in x_test:
    predictions.append(face_recognizer.predict(i))

predictions = np.array(predictions)
y_test = np.array(y_test)
accuracy = np.sum(np.abs(predictions[:, 0]==y_test))/len(predictions)
print("fisherface accuracy:")
print(accuracy)

fisherface accuracy:
0.6363636363636364


In [25]:

# if 'EIGENFACE' not in os.listdir(r'E:\semester 5\mini project\face recognition algorithms'):
face_recognizer = cv2.face.EigenFaceRecognizer_create()
face_recognizer.train(x_train,y_train)
face_recognizer.write('EIGENFACE')

# else :
#     face_recognizer = cv2.face.EigenFaceRecognizer_create()
#     face_recognizer.read('EIGENFACE')

predictions = []
for i in x_test:
    predictions.append(face_recognizer.predict(i))

predictions = np.array(predictions)
y_test = np.array(y_test)
accuracy = np.sum(np.abs(predictions[:, 0]==y_test))/len(predictions)
print("eigenface accuracy:")
print(accuracy)

eigenface accuracy:
0.5454545454545454


In [26]:
x,y = make_dataset()
x_train ,x_test, y_train, y_test = train_test_split(x,y,test_size=0.10, random_state=60)
x_train = preprocess_data(x_train, True, False)
x_test = preprocess_data(x_test, True, False)

# if 'LBPH' not in os.listdir(r'E:\semester 5\mini project\face recognition algorithms'):
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.train(x_train,y_train)
face_recognizer.write('LBPH')

# else :
#     face_recognizer = cv2.face.LBPHFaceRecognizer_create()
#     face_recognizer.read('LBPH')

# test_img = r"E:\semester 5\mini project\face recognition algorithms\dataset\Hrishi\20.jpg"
# image = cv2.imread(test_img)
# faces = get_faces_from_image_2(image)


# if len(faces):
#     index = 0
#     # plt.imshow(faces[index][:,:,::-1])
#     face = cv2.cvtColor(faces[index], cv2.COLOR_BGR2GRAY)
#     face = cv2.resize(face, SIZE)
#     plt.imshow(face,cmap='gray')
#     prediction, confidence = face_recognizer.predict(face)
#     print([k for k,v in names.items() if v==prediction])
#     print("confidence :", confidence)
#     plt.show()

predictions = []
for i in x_test:
    predictions.append(face_recognizer.predict(i))

predictions = np.array(predictions)
y_test = np.array(y_test)
accuracy = np.sum(np.abs(predictions[:, 0]==y_test))/len(predictions)
print("lbph accuracy:")
print(accuracy)

lbph accuracy:
0.9090909090909091


In [27]:
img = cv2.imread(r"E:\semester 5\mini project\face recognition algorithms\dataset\Tomar\111.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
print([k for k,v in names.items() if v==face_recognizer.predict(img)[0]])

['Tomar']
